# 🌍 Global Sales Performance Dashboard

This notebook provides an interactive dashboard for analyzing global sales performance across continents and regions.

## Features:
- Geographic sales visualization with interactive world maps
- Regional comparison charts and trends
- Top performer analysis
- Key performance indicators
- Interactive filters for deep-dive analysis

## Setup and Data Loading

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Import our custom modules
import sys
sys.path.append('../src')
from data_processor import SalesDataProcessor, create_sample_data
from visualizations import SalesVisualizer

print("Libraries imported successfully! ✅")

In [ ]:
# Initialize components
processor = SalesDataProcessor()
visualizer = SalesVisualizer()

# Load and process data
print("Processing sales data...")
data_dict = processor.process_full_pipeline('../data/sample_sales_data.csv')

print(f"✅ Data loaded successfully!")
print(f"📊 Total records: {len(data_dict['cleaned_data']):,}")
print(f"🌍 Countries: {len(data_dict['country_data'])}")
print(f"🗺️  Regions: {len(data_dict['continent_data'])}")
print(f"📅 Year range: {data_dict['cleaned_data']['Year'].min()} - {data_dict['cleaned_data']['Year'].max()}")

## Key Performance Indicators

In [ ]:
# Calculate and display KPIs
kpis = visualizer.create_kpi_cards(data_dict)

def format_number(num):
    if num >= 1e9:
        return f"${num/1e9:.2f}B"
    elif num >= 1e6:
        return f"${num/1e6:.2f}M"
    elif num >= 1e3:
        return f"${num/1e3:.0f}K"
    else:
        return f"${num:.0f}"

print("📈 KEY PERFORMANCE INDICATORS")
print("=" * 50)
print(f"💰 Total Sales: {format_number(kpis.get('total_sales', 0))}")
print(f"💵 Total Profit: {format_number(kpis.get('total_profit', 0))}")
print(f"📊 Profit Margin: {kpis.get('profit_margin', 0):.1f}%")
print(f"🌍 Countries: {kpis.get('total_countries', 0):,}")
print(f"🗺️  Regions: {kpis.get('total_regions', 0):,}")
print(f"📈 Growth Rate: {kpis.get('growth_rate', 0):.1f}%")
print(f"🎯 Avg Sales/Country: {format_number(kpis.get('avg_sales_per_country', 0))}")

## Geographic Sales Distribution

In [ ]:
# Create and display world map
print("🗺️ Creating interactive world map...")
world_map = visualizer.create_world_map(data_dict['country_data'])
world_map.show()

## Regional Analysis

In [ ]:
# Regional bar chart and pie chart
fig_bar = visualizer.create_continent_bar_chart(data_dict['continent_data'])
fig_pie = visualizer.create_sales_distribution_pie(data_dict['continent_data'])

print("📊 Regional Sales Analysis")
fig_bar.show()
fig_pie.show()

In [ ]:
# Regional summary table
print("📋 Regional Summary Table")
regional_summary = data_dict['continent_data'].copy()
regional_summary['Total_Sales'] = regional_summary['Total_Sales'].apply(lambda x: f"${x:,.0f}")
regional_summary['Average_Sales'] = regional_summary['Average_Sales'].apply(lambda x: f"${x:,.0f}")
regional_summary['Total_Profit'] = regional_summary['Total_Profit'].apply(lambda x: f"${x:,.0f}")

display(regional_summary)

## Sales Trends Over Time

In [ ]:
# Growth trends chart
print("📈 Creating sales trends chart...")
if not data_dict['growth_trends'].empty:
    growth_chart = visualizer.create_growth_trend_chart(data_dict['growth_trends'])
    growth_chart.show()
    
    # Show yearly summary
    yearly_summary = data_dict['growth_trends'].groupby('Year')['Sales'].sum().reset_index()
    yearly_summary['Sales'] = yearly_summary['Sales'].apply(lambda x: f"${x:,.0f}")
    print("\n📅 Yearly Sales Summary:")
    display(yearly_summary)
else:
    print("⚠️ No time series data available for trend analysis")

## Top Performers Analysis

In [ ]:
# Top countries chart
print("🏆 Top Performing Countries")
top_countries_chart = visualizer.create_top_performers_chart(
    data_dict['top_countries'], 
    'Top 10 Countries by Sales', 
    'Country', 
    'Total_Sales'
)
top_countries_chart.show()

In [ ]:
# Profit vs Sales relationship
print("💰 Sales vs Profit Analysis")
profit_scatter = visualizer.create_profit_vs_sales_scatter(data_dict['country_data'])
profit_scatter.show()

In [ ]:
# Top performers table
print("📊 Top 15 Countries Performance Table")
top_performers = data_dict['country_data'].head(15).copy()
top_performers['Total_Sales'] = top_performers['Total_Sales'].apply(lambda x: f"${x:,.0f}")
top_performers['Average_Sales'] = top_performers['Average_Sales'].apply(lambda x: f"${x:,.0f}")
top_performers['Total_Profit'] = top_performers['Total_Profit'].apply(lambda x: f"${x:,.0f}")

# Calculate profit margin
original_data = data_dict['country_data'].head(15)
top_performers['Profit_Margin'] = (original_data['Total_Profit'] / original_data['Total_Sales'] * 100).apply(lambda x: f"{x:.1f}%")

display(top_performers)

## Interactive Filtering (Optional)

In [ ]:
# Interactive widgets for filtering (requires ipywidgets)
try:
    from ipywidgets import interact, SelectMultiple, IntSlider
    
    @interact
    def filter_and_display(
        regions=SelectMultiple(
            options=list(data_dict['cleaned_data']['Region'].unique()),
            value=list(data_dict['cleaned_data']['Region'].unique()),
            description='Regions:'
        ),
        year_range=IntSlider(
            min=data_dict['cleaned_data']['Year'].min(),
            max=data_dict['cleaned_data']['Year'].max(),
            value=data_dict['cleaned_data']['Year'].max(),
            description='Latest Year:'
        )
    ):
        # Filter data based on selections
        filtered_data = data_dict['cleaned_data'][
            (data_dict['cleaned_data']['Region'].isin(regions)) &
            (data_dict['cleaned_data']['Year'] <= year_range)
        ]
        
        if not filtered_data.empty:
            # Show filtered summary
            total_sales = filtered_data['Sales'].sum()
            total_profit = filtered_data['Profit'].sum()
            countries = filtered_data['Country'].nunique()
            
            print(f"📊 Filtered Results:")
            print(f"💰 Total Sales: {format_number(total_sales)}")
            print(f"💵 Total Profit: {format_number(total_profit)}")
            print(f"🌍 Countries: {countries}")
            
            # Quick regional breakdown
            regional_breakdown = filtered_data.groupby('Region')['Sales'].sum().sort_values(ascending=False)
            print(f"\n🗺️ Regional Breakdown:")
            for region, sales in regional_breakdown.items():
                print(f"  {region}: {format_number(sales)}")
        else:
            print("⚠️ No data matches the selected filters")
            
except ImportError:
    print("💡 Install ipywidgets for interactive filtering: pip install ipywidgets")
    print("    Then restart the kernel and run this cell again.")

## Data Export Options

In [ ]:
# Export options
print("💾 Data Export Options")
print("=" * 30)

# Export regional summary
data_dict['continent_data'].to_csv('../data/regional_summary.csv', index=False)
print("✅ Regional summary exported to: ../data/regional_summary.csv")

# Export country summary
data_dict['country_data'].to_csv('../data/country_summary.csv', index=False)
print("✅ Country summary exported to: ../data/country_summary.csv")

# Export growth trends (if available)
if not data_dict['growth_trends'].empty:
    data_dict['growth_trends'].to_csv('../data/growth_trends.csv', index=False)
    print("✅ Growth trends exported to: ../data/growth_trends.csv")

print("\n📁 All exports saved to the data folder.")

## Custom Analysis Section

In [ ]:
# Add your custom analysis here
print("🔍 Custom Analysis Section")
print("This cell is for your own custom analysis and visualizations.")
print("You can access all the processed data through the 'data_dict' variable.")
print("\nAvailable data keys:")
for key in data_dict.keys():
    print(f"  - {key}: {type(data_dict[key]).__name__} with shape {data_dict[key].shape if hasattr(data_dict[key], 'shape') else 'N/A'}")

# Example custom analysis:
# Find the most profitable product category by region
if 'Product_Category' in data_dict['cleaned_data'].columns:
    product_analysis = data_dict['cleaned_data'].groupby(['Region', 'Product_Category']).agg({
        'Sales': 'sum',
        'Profit': 'sum'
    }).reset_index()
    
    print("\n🛍️ Most Profitable Product by Region:")
    for region in product_analysis['Region'].unique():
        region_data = product_analysis[product_analysis['Region'] == region]
        top_product = region_data.loc[region_data['Profit'].idxmax()]
        print(f"  {region}: {top_product['Product_Category']} (${top_product['Profit']:,.0f} profit)")

---

## 👨‍💻 Developer

**Developed By:** Mahender Banoth (IIT Patna) 

<div style="display: flex; align-items: center; gap: 10px; margin-top: 10px;">
    <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/a/a5/Instagram_icon.png/1024px-Instagram_icon.png" width="20" height="20" alt="Instagram">
    <span style="font-weight: 600; color: #E4405F;">@mahender_hustles</span>
</div>

---